# Assignment 5: Pratical Machine Learning

## Overview

Imagine you want to build a classification model for your company. It's not uncommon that you may encounter the following three problems:

* There is no training data available; data labeling may cost you a lot of money and time.
* The number of features is very large, which may not only lead to long training time but also hurt model generalization. 
* There are a number of parameters associated with your model, and you do not know what parameters should be used. 

In this assignment, you will learn how to use Active Learning to address the first problem, learn how to apply a feature selection approach as well as Principal Component Analysis (PCA) to addressing the second one, and learn how to use the Spark ML pipeline API to tune parameters.


After completing this assignment, you should be able to answer the following questions:

**Data Labeling**
1. Why Active Learning?
2. How to implement uncertain sampling, a popular query strategy for Active Learning?
3. How to solve an ER problem using Active Learning?

**Feature Selection**
1. Why Feature Selection?
2. What are typical Feature Selection approaches?
3. How does a filter-based feature selection approach work?
4. How does PCA work?
5. What're the advantages and disadvantages of PCA? 

**Parameter Tuning**
1. Why parameter tuning?
2. What is cross-validation?
3. What is areaUnderROC?
4. How to tune parameters?




Note: In this assignment, the example code was writtin for Python 2. If you use Python 3, please feel free to make changes (e.g., print x --> print(x)).

You can download the datasets of each assignment from http://tiny.cc/cmpt733-datasets.

## Part 1. Data Labeling

[Active learning](http://tiny.cc/al-wiki) is a certain type of ML algorithms that can train a high-quality ML model with small data-labeling cost. Its basic idea is quite easy to understand. Consider a typical supervised ML problem, which requires a (relatively) large training dataset. In the training dataset, there may be only a small number of data points that are beneficial to the trained ML model. In other words, labeling the small number of data points is enough to train a high-quality ML model. The goal of active learning is to help us to identify the data points. 


In this assignment, we will revisit [Entity Resolution](https://sfu-db.github.io/bigdata-cmpt733/Lectures/lec2.pdf) and develop an Active Learning approach for it. Recall that entity Resolution (ER) is defined as finding different records that refer to the same real-world entity, e.g., "iPhone 4-th Generation" vs. "iPhone Four". It is central to data integration and cleaning. The following figure shows the architecture of an entity resolution solution. It consists of four major steps. **I will provide you the source code for Steps 1, 2, 4. Your job is to implement Step 3.**  

<img src="img/arch.png", width=800/>

### Step 1. Read Data

The [restaurant data](restaurant.csv) is in an CSV file. Here is the code to read it. 

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('restaurant.csv')
print("{} :".format(df.shape))
df.head(5)

(858, 5) :


,id,name,address,city,type
0,1,arnie morton's of chicago,435 s. la cienega blv.,los angeles,american
1,2,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,steakhouses
2,3,art's delicatessen,12224 ventura blvd.,studio city,american
3,4,art's deli,12224 ventura blvd.,studio city,delis
4,5,hotel bel-air,701 stone canyon rd.,bel air,californian


From this dataset, you will find many matching record pairs. For example, the first two rows shown above are matching (i.e., refer to the same real-world entity). You can check out all matching record pairs in the [true_matches.json](true_matches.json) file. 

### Step 2. Remove Obviously Non-matching Pairs

A naive implementation of entity resolution is to enumerate all $n^2$ pairs and check whether they are matching or not. As you've learnt in [Assignment 2](https://rawgit.com/sfu-db/bigdata-cmpt733/master/Assignments/A2/A2.html), we can avoid $n^2$ comparisons using a similarity-join algorithm. 

Here is the code. After running the code, you will get 678 similar pairs ordered by their similarity decreasingly.

In [2]:
from a9_utils import *

data = df.values.tolist()
simpairs = simjoin(data)

print("Num of Pairs:{} ".format(len(data)*(len(data)-1)/2))
print("Num of Similar Pairs:{} ".format(len(simpairs)))
print("The Most Similar Pair: {}".format(simpairs[0]))

Num of Pairs:367653.0 
Num of Similar Pairs:678 
The Most Similar Pair: ([170, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern/soul'], [169, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern'])


We can see that `simjoin` helps us remove the number of pairs from 367653 to 678. But, there are still many non-matching pairs in `simpairs` (see below). 

In [3]:
print(simpairs[-1])

([764, "buzio's in the rio", '3700 w. flamingo rd.', 'las vegas', 'seafood'], [542, 'carnival world', '3700 w. flamingo rd.', 'las vegas', 'buffets'])


Next, we will use active learning to train a classifier, and then use the classifier to classify each pair in `simpairs` as either "matching" or "nonmatching". 

### Step 3. Active Learning (Task A)

Given a set of similar pairs, what you need to do next is to iteratively train a classifier to decide which pairs are truly matching. We are going to use [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) as our classifier. 

#### Featurization

To train the model, the first thing you need to think about is how to featurize data. That is, transforming each similar pair to a feature vector. Please use the `featurize()` function for featurization. The function takes a pair as input and returns a vector of 6 features.

In [4]:
from a9_utils import featurize

print("The feature vector of the first pair:{} ".format(featurize(simpairs[0])))

The feature vector of the first pair:(1.0, 1.0, 1.0, 1.0, 1.0, 0.6153846153846154) 


#### Initialization

At the beginning, all the pairs are unlabeled. To initialize a model, we first pick up ten pairs and then label each pair using  the `crowdsourcing()` function. You can assume that `crowdsourcing()` will ask a crowd worker (e.g., on Amazon Mechanical Turk) to label a pair. 


`crowdsourcing(pair)` is a function that simulates the use of crowdsourcing to label a pair
  
  - **Input:**	pair – A pair of records 

  - **Output:**	Boolean –  *True*: The pair of records are matching; *False*: The pair of records are NOT matching;

Please use the following code to do the initialization. 

In [6]:
from a9_utils import crowdsourcing

# choose the most/least similar five pairs as initial training data
init_pairs = simpairs[:5] + simpairs[-5:]
matches = []
nonmatches = []
for pair in init_pairs:
    is_match = crowdsourcing(pair)
    if is_match == True:
        matches.append(pair)
    else:
        nonmatches.append(pair)
        
print("Number of matches:{}".format(len(matches)))
print("Number of nonmatches:{}".format(len(nonmatches)))

Are they matching?
[170, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern/soul']
[169, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern']
Answer: Yes
Are they matching?
[88, 'manhattan ocean club', '57 w. 58th st.', 'new york city', 'seafood']
[87, 'manhattan ocean club', '57 w. 58th st.', 'new york', 'seafood']
Answer: Yes
Are they matching?
[112, 'san domenico', '240 central park s.', 'new york city', 'italian']
[111, 'san domenico', '240 central park s', 'new york', 'italian']
Answer: Yes
Are they matching?
[197, 'fleur de lys', '777 sutter st.', 'san francisco', 'french (new)']
[196, 'fleur de lys', '777 sutter st.', 'san francisco', 'french']
Answer: Yes
Are they matching?
[8, 'cafe bizou', '14016 ventura blvd.', 'sherman oaks', 'french bistro']
[7, 'cafe bizou', '14016 ventura blvd.', 'sherman oaks', 'french']
Answer: Yes
Are they matching?
[709, 'arcadia', '21 e. 62nd st.', 'new york city', 'american (new)']
[66, 'four seasons', '99 e. 5

#### Interactive Model Training

Here is the only code you need to write in Part 1.


In [98]:
from a9_utils import featurize, crowdsourcing
import random

labeled_pairs = matches + nonmatches
unlabeled_pairs = [p for p in simpairs if p not in labeled_pairs]
iter_num = 5

#<-- Write Your Code -->
from sklearn.linear_model import LogisticRegression as LR
#initialize model 
model = LR(solver='lbfgs')
X_train = [list(featurize(labeled_pairs[features])) for features in range(len(labeled_pairs))]
Y_train = list(np.ones(5))+list(np.zeros(5)) #Creating prediction variable for the model.

def train_model(model,X_train,Y_train,unlabeled_pair):
    model.fit(X_train,Y_train)
    X_pred = [list(featurize(unlabeled_pair[features])) for features in range(len(unlabeled_pair))]
    prediction = model.predict(X_pred)
    probability = model.predict_proba(X_pred)
    x = probability[:,0]
    indexes = np.where(x==min(x))
    index = random.choice(indexes[0])
    return index
    
    
    

for i in range(iter_num):
    uncertain_index = train_model(model,X_train,Y_train,unlabeled_pairs)
    uncertain_pair = unlabeled_pairs.pop(index)
    is_match = 1 if crowdsourcing(uncertain_pair) else 0
    X_train.append(featurize(uncertain_pair))
    Y_train.append(is_match)

print(len(labeled_pairs))
print(len(unlabeled_pairs))

Are they matching?
[64, 'felidia', '243 e. 58th st.', 'new york city', 'italian']
[63, 'felidia', '243 e. 58th st.', 'new york', 'italian']
Answer: Yes
Are they matching?
[98, 'oceana', '55 e. 54th st.', 'new york city', 'seafood']
[97, 'oceana', '55 e. 54th st.', 'new york', 'seafood']
Answer: Yes
Are they matching?
[104, 'picholine', '35 w. 64th st.', 'new york city', 'mediterranean']
[103, 'picholine', '35 w. 64th st.', 'new york', 'mediterranean']
Answer: Yes
Are they matching?
[185, 'alain rondelli', '126 clement st.', 'san francisco', 'french (new)']
[184, 'alain rondelli', '126 clement st.', 'san francisco', 'french']
Answer: Yes
Are they matching?
[193, 'campton place', '340 stockton st.', 'san francisco', 'american (new)']
[192, 'campton place', '340 stockton st.', 'san francisco', 'american']
Answer: Yes
10
663


**[Algorithm Description].**   Active learning has many [query strategies](http://tiny.cc/al-wiki-qs) to decide which data point should be labeled. You need to implement uncertain sampling for Task A. The algorithm trains an initial model on `labeled_pairs`. Then, it iteratively trains a model. At each iteration, it first applies the model to `unlabeled_pairs`, and makes a prediction on each unlabeled pair along with a probability, where the probability indicates the confidence of the prediction. After that, it selects the most uncertain pair (If there is still a tie, break it randomly),  and call the `crowdsroucing()` function to label the pair. After the pair is labeled, it updates `labeled_pairs` and `unlabeled_pairs`, and then retrain the model on `labeled_pairs`.

**[Input].** 
- `labeled_pairs`: 10 labeled pairs (by default)
- `unlabeled_pairs`: 668 unlabeled pairs (by default)
- `iter_num`: 5 (by default)

**[Output].** 
- `model`: A logistic regression model built by scikit-learn


### Step 4. Model Evaluation

[Precision, Recall](https://en.wikipedia.org/wiki/Precision_and_recall), [F-Score](https://en.wikipedia.org/wiki/F1_score) are commonly used to evaluate an entity-resolution result. After training an model, you can use the following code to evalute it.

In [100]:
import json
import numpy as np
from a9_utils import evaluate

            
sp_features = np.array([featurize(sp) for sp in simpairs])
label = model.predict(sp_features)
pair_label = zip(simpairs, label)

identified_matches = []
for pair, label in pair_label:
    if label == 1:
        identified_matches.append(pair)
        
precision, recall, fscore = evaluate(identified_matches)

print("Precision:{}".format(precision))
print("Recall:{}".format(recall))
print("Fscore:{}".format(fscore))
   

Precision:0.8888888888888888
Recall:0.9056603773584906
Fscore:0.897196261682243


## Part 2. Feature Selection

Feature selection and dimensionality reduction have been extensively studied in the ML community. In this part, I designed two simple tasks for you to get familiar with these topics. If you want to know more, please refer to Prof. Michael Jordan's [Practical Machine Learning](https://people.eecs.berkeley.edu/~jordan/courses/294-fall09/) course at UC Berkeley. 

### Task B. Filter-based Feature Selection

There are [three classes](https://en.wikipedia.org/wiki/Feature_selection#Main_principles) of feature-selection methods: filter-based, wrapper-based, and embeded-based. Filter-based is the most simple one. Its basic idea is to assign a heuristic score to each feature to filter out the "obviously" useless ones. There are many ways to compute the score, e.g., chi-squared, information gain, correlation, and mutual information. We use [chi-squared](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html) for Task B. 

Imagine you have a collection of newsgroup documents, and you want to build a classification model to predicate the topic of each newsgroup document: "comp.graphics (0)" or "sci.space (1)". You use bag of words to represent each document. That is, each feature is a single word. I have already helped you load the dataset and generated the feature vectors (see the code below). 

In [103]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer


data_train = fetch_20newsgroups(subset='train', categories=['comp.graphics','sci.space'],
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_train.data)
y = data_train.target

print("Target Labels:{}".format(data_train.target_names))
print("(NSamples, NFeatures):{}".format(X.shape))


Target Labels:['comp.graphics', 'sci.space']
(NSamples, NFeatures):(1177, 19493)


From the output of the above code, you find that the number of features (19493) is very large, so you want to use a filter-based method to choose the top-10 words as your features. 

Please write your code below. The code computes chi-squared stat between each feature and the target label, and return the top-10 words with the highest score. 

In [120]:
from sklearn.feature_selection import SelectKBest, chi2

#<-- Write Your Code -->
chi_score,pvals = chi2(X,y)
X_new = SelectKBest(chi2,k=10).fit(X,y)
print(type(X_new))
indexes = X_new.get_support(indices=True)

# retrieve top features
features = vectorizer.get_feature_names()
top_features = [features[i] for i in indexes]
print(top_features)


<class 'sklearn.feature_selection.univariate_selection.SelectKBest'>
['3d', 'file', 'files', 'graphics', 'image', 'moon', 'nasa', 'orbit', 'space', 'windows']


### Task C. Principal Component Analysis (PCA)

PCA is a well-known dimensionality reduction algorithm. Obviously, we can use PCA to transform each feature vector to a 10-D feature vector. See the code below. 

In [116]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca.fit(X.toarray())
print(pca.explained_variance_ratio_)

[0.01690229 0.00799231 0.00645212 0.00520748 0.00477457 0.00446175
 0.0041662  0.00391556 0.00382357 0.00365613]


Both filter-based feature selection and PCA can help us reduce the number of features in a feature vector. The question is what's the difference between them. In this task, you don't need to write any code. Just let me know what you think about the advantages and disadvantages of PCA compared to filter-based feature section. Please list (at least) two advantages and two disadvantages. 

**Advantages of PCA**
 - Easy to use, extracts the top 10 features based on variance calculated for each features which is good for large datasets with varying features.  
 - considers all the features for calculation unlike chi squared technique which uses only individual features. 

**Disadvantages of PCA**
- Does not provide the flexibility to choose the "scoring" function. Only uses the method of variance. Sometimes variance is not the most optimal method of feature extraction. For ex. smaller datasets.  
- Slower computation than other methods as it finds the principal component based on all the features. 

## Part 3. Parameter Tuning

### Preliminary

Imagine you have a collection of newsgroup documents, and you want to build a classification model to predicate the topic of each newsgroup document: "science (1)" or "non-science (0)". Here is the Spark program (below) that can help you to finish the job. ** Your preliminary task is to read the code and understand how it works. ** I highly recommend you to read through the [Spark ML Pipeline Programming Guide](http://spark.apache.org/docs/latest/ml-pipeline.html), which is very well written, and will aid your understanding of the code. 

In [130]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName('parameter tuning').getOrCreate()
sc = spark.sparkContext
trainDF = spark.read.parquet("20news_train.parquet")

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features", numFeatures=1000)
lr = LogisticRegression(maxIter=20, regParam=0.1)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training data.
model = pipeline.fit(trainDF)

# Evaluate the model on testing data
testDF = spark.read.parquet("20news_test.parquet")
prediction = model.transform(testDF)
evaluator = BinaryClassificationEvaluator()
print(evaluator.evaluate(prediction))


0.7486382404197034


If you run the program, you will see that the trained model can only get an [areaUnderROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) of ~0.75 on the testing dataset.

### Task D - Parameter Tuning

Now it comes to the final task of the assignment. Let's take a look at the above program. It actually used `numFeatures=1000` and `regParam=0.1` to train the model. One natural question is that if we used different values for the two parameters, would that lead to a better model (i.e., a higher areaUnderROC)?


In this task, your job is to complete the following code. 

In [132]:
# The code will tune the following two parameters on the training dataset:
# 
#   numFeatures=1000, 5000, 10000
#   regParam=0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
# 
# and will use 2-fold cross-validation for model evaluation.

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=20)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

paramgrid = ParamGridBuilder().addGrid(hashingTF.numFeatures,[1000, 5000, 10000]) \
                            .addGrid(lr.regParam,[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) \
                            .build()
cross_validator = CrossValidator(estimator=pipeline,
                                 estimatorParamMaps=paramgrid,
                                 evaluator=BinaryClassificationEvaluator(),
                                 numFolds=2)


cv_model = cross_validator.fit(trainDF)
prediction = cv_model.transform(testDF)
evaluator = BinaryClassificationEvaluator()
print(evaluator.evaluate(prediction))

0.8536581733181826


Once you get the best model with the best parameters, please apply the model to the testing dataset, and `print` the new areaUnderROC value. 

## Submission

Download [A5.zip](A5.zip). Complete Tasks A, B, C and D in A5.ipynb, and submit it to the CourSys activity Assignment 5.